In [ ]:
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime

# === Path settings ===
main_file   = r"C:\Users\51195\Desktop\MR链式表.xlsx"
trait_file  = r"C:\Users\51195\Desktop\Trait(最终完整版包含更新GWAS所有数据).xlsx"
output_file = r"C:\Users\51195\Desktop\MR链式表带注释.xlsx"

# === Read data (all as strings, keep "NA" text) ===
df_main  = pd.read_excel(
    main_file,
    keep_default_na=False,
    na_values=[],
    dtype=str,
    engine="openpyxl"
)
trait_df = pd.read_excel(
    trait_file,
    keep_default_na=False,
    na_values=[],
    dtype=str,
    engine="openpyxl"
)

# === Building a mapping dictionary ===
chinese_map = trait_df.set_index('ID')['chinese'].to_dict()
trait_map   = trait_df.set_index('ID')['trait'].to_dict()

# === Defines the ID pattern to match ===
# Among them, finngen[-_] matches both finngen-xxx and finngen_xxx
pattern = re.compile(
    r"(GCST\d+|ieu-[\w\-_]+|met-[\w\-_]+|prot-[\w\-_]+|bbj-[\w\-_]+|ubm-[\w\-_]+|finngen[-_][\w\-_]+)"
)

# === Annotation function (vectorized) ===
def annotate_series(series: pd.Series) -> pd.Series:
    # Extract the first matching ID
    ids = series.str.extract(pattern, expand=False)
    # Mapping Chinese and English annotations
    chinese = ids.map(chinese_map).fillna("")
    trait   = ids.map(trait_map).fillna("")
    # Splicing annotation: ID\tChinese\tEnglish
    annotation = ids + "\t" + chinese + "\t" + trait
    # Conditional replacement: use comments when matching, otherwise keep the original value
    return np.where(ids.notna(), annotation, series)

# === Apply comments to all columns ===
for col in df_main.columns:
    df_main[col] = annotate_series(df_main[col])

# === Make sure the output directory exists ===
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# === Save function, if PermissionError is encountered, save a new file with timestamp ===
def save_with_fallback(df, path):
    try:
        df.to_excel(path, index=False, engine="openpyxl")
        print(f"文件已保存到：{path}")
    except PermissionError:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base, ext = os.path.splitext(path)
        new_path = f"{base}_{timestamp}{ext}"
        df.to_excel(new_path, index=False, engine="openpyxl")
        print(f"原文件无法写入，已另存为：{new_path}")

# === Execute Save ===
save_with_fallback(df_main, output_file)
